Skeleton Code

In [2]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-2-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-2-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [5]:
df.shape

(7352, 561)

In [6]:
df.describe()


,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,0.125293,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.250994,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.023692,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,0.134000,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,0.289096,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000


In [6]:
df.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
556,0
557,0
558,0
559,0


In [7]:
correlation_matrix = df.corr()

In [10]:
print(correlation_matrix)

          0         1         2         3         4         5         6    \
0    1.000000  0.148061 -0.256952  0.000619 -0.021903 -0.044617  0.006290   
1    0.148061  1.000000 -0.078769 -0.045160 -0.044920 -0.049746 -0.044180   
2   -0.256952 -0.078769  1.000000 -0.020217 -0.016641 -0.008410 -0.018747   
3    0.000619 -0.045160 -0.020217  1.000000  0.927461  0.851668  0.998632   
4   -0.021903 -0.044920 -0.016641  0.927461  1.000000  0.895510  0.922803   
..        ...       ...       ...       ...       ...       ...       ...   
556  0.037444  0.017967 -0.063609  0.018985 -0.008507 -0.018429  0.019389   
557  0.028844  0.075679 -0.034037 -0.024810 -0.014592 -0.006471 -0.024951   
558 -0.035257 -0.005309  0.008587 -0.371653 -0.380531 -0.345011 -0.368191   
559  0.034371  0.001053 -0.015288  0.471065  0.523600  0.476006  0.466424   
560  0.028242 -0.013903 -0.022643  0.394825  0.433169  0.482828  0.390922   

          7         8         9    ...       551       552       553  \
0  

In [8]:
df.nunique()

,0
0,7347
1,7352
2,7349
3,7349
4,7351
...,...
556,7352
557,7352
558,7352
559,7352


In [9]:
# Solution to Task 2: Encode class labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())



In [10]:
#Scale the features using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)



In [11]:
#Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled, encoded_y, test_size=0.2, random_state=42)

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# 1. Create a pipeline using Gaussian Naive Bayes
pipeline = Pipeline([
    ('naive_bayes', GaussianNB())
])

# 2. Fit the model to the training data
pipeline.fit(X_train_full, y_train)

# 3. Predict values for the test set
y_pred = pipeline.predict(X_test_full)

# 4. Print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7314751869476547


In [13]:
import time

# 1. Note the start time before defining the pipeline
start_time = time.time()

# 2. Note the end time and report the difference
end_time = time.time()
time_taken = end_time - start_time
print(f"Time taken: {time_taken} seconds")


Time taken: 4.1961669921875e-05 seconds


In [14]:
# Define a Pipeline with Gaussian Naive Bayes and train it on the full dataset
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train_full, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test_full)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)

In [17]:
#K-Means for dimensionality reduction
n_clusters = 20  # Choose the desired number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)  # Initialize KMeans
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = []
selected_features = df.iloc[:, selected_features_indices]


In [20]:
#Train another model (GaussianNB) on the new dataset, and report time taken and accuracy
# One approach is to pick one feature from each cluster:
selected_features_indices = [np.random.choice(np.where(kmeans.labels_==i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:,selected_features_indices]

# Now we can proceed with model training on the reduced dataset
X_train, X_test, y_train, y_test = train_test_split(selected_features, encoded_y, test_size=0.2, random_state=42)

# Scale the selected features
scaler_reduced = StandardScaler()
X_train = scaler_reduced.fit_transform(X_train)
X_test = scaler_reduced.transform(X_test)

# Gaussian Naive Bayes pipeline
start_time_reduced = time.time()  # Record start time
classifier_pipeline_reduced = Pipeline([('classifier', GaussianNB())])
classifier_pipeline_reduced.fit(X_train, y_train)
y_pred_reduced = classifier_pipeline_reduced.predict(X_test)
end_time_reduced = time.time()  # Record end time

# Calculate and report time and accuracy
reduced_features_time = end_time_reduced - start_time_reduced
accuracy_reduced = accuracy_score(y_test, y_pred_reduced)

print(f"Time taken with reduced features: {reduced_features_time:.4f} seconds")
print(f"Accuracy with reduced features: {accuracy_reduced:.4f}")


Time taken with reduced features: 0.0078 seconds
Accuracy with reduced features: 0.7254
